# Best Practices for Prompt Engineering

source: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


In [1]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT


CHAT_COMPLETIONS_MODEL = os.getenv('DEPLOYMENT_NAME')

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [3]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Summarize the text below as a bullet point list of the most important points. \n\n \
        ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])

- OpenAI and Microsoft are extending their partnership.
- Microsoft is making a multi-year, multi-billion dollar investment in OpenAI.
- The investment will support independent research and the development of safer, more useful, and more powerful AI.
- OpenAI remains a capped-profit company, governed by the non-profit organization OpenAI.
- The partnership between OpenAI and Microsoft aligns with their shared vision and values.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [5]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])


In a realm where ideas dance and float,
Lies a wonder, OpenAI's boat.
A vision inspiring minds to soar,
Unveiling possibilities, forevermore.

The fusion of science and art collide,
OpenAI's brilliance, a flawless guide.
Through algorithms and data's might,
Unleashing innovation, like stars at night.

From language to images, it breaks the mold,
A tapestry woven with stories untold.
Transforming words into meaningful verse,
Unlocking the power of the universe.

From chatbots wise, with knowledge vast,
To creative tools that leave us aghast.
OpenAI's magic, a creative force,
Reshaping the world with a boundless source.

From deep learning's depths, it does arise,
A symphony of knowledge, reaching the skies.
With neural networks, it's forged in fire,
Pushing boundaries higher and higher.

In realms of research and AI's domain,
OpenAI paves the way, brave and insane.
With ethics and fairness leading the way,
Guiding the revolution, day after day.

OpenAI, a beacon shining bright,
Illumina

In [7]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])



In the realm of human thought,
Where innovation battles naught,
There emerged a digital force,
OpenAI, a mighty source.

With pixels as its faithful guide,
DALL-E arose, standing worldwide.
In Hemingway's shadow, it found its voice,
Inspiring minds with timeless choice.

Creativity, unleashed, it seemed,
As DALL-E's art, like dreams redeemed.
Imagination conquered the unknown,
Through vibrant images it had sewn.

Like scribbles brushed upon a page,
DALL-E dared, it dared engage,
With algorithms finely tuned,
A bridge between the artist's swoon.

No longer bound by human hands,
OpenAI's creation expands,
Where endless crafts of new design,
Emerge from codes with strength divine.

So let us heed the whispered call,
Where human spirit meets the all,
With OpenAI's seeds sown deep,
In every heart, innovation shall reap.


# 4. Articulate the desired output format through examples (example 1, example 2). 

In [8]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the companyn names then years in the following text below and output start index and end index of each entity.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                ###\
                ',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])



[{"text": "OpenAI", "start": 98, "end": 104}, {"text": "Microsoft", "start": 113, "end": 122}, {"text": "2019", "start": 207, "end": 211}, {"text": "2021", "start": 216, "end": 220}]


In [9]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the entities mentioned in the text below. \
                Extract the important entities mentioned in the text below. \
                First extract all company names, then extract all years, \
                then extract specific topics which fit the content and finally extract general overarching themes\n\n \
                Desired format: \
                Company names: <comma_separated_list_of_company_names> \
                Years: -||- \
                Specific topics: -||- \
                General themes: -||- \
                """\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                """\
                ',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])



Company names: OpenAI, Microsoft

Years: 2019, 2021

Specific topics: independent research, AI safety, AI usefulness, AI power

General themes: partnership, investment, continuation of research and development


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [13]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])



OpenAI, Microsoft, partnership, investment, multi-year, multi-billion dollar, independent research, AI, safe, useful, powerful


In [14]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below.\n\n \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])



OpenAI, Microsoft, partnership, investment, multi-year, multi-billion dollar, independent research, AI, safe, useful, powerful.


In [9]:
response = openai.Completion.create(
            engine=os.getenv('DEPLOYMENT_NAME'),
            prompt='Extract keywords from the corresponding texts below.\n\n \
                Text 1: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords 1: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ##\n\
                Text 2: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords 2: OpenAI, language models, text processing, API.\n\n\
                ##\n\
                Text 3: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords 3:',
            max_tokens=400,
            n=1
        )

print(response['choices'][0]['text'])

 OpenAI, Microsoft, partnership, investment, research, AI.


# 6. Reduce “fluffy” and imprecise descriptions

In [16]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])



Introducing the next generation of car seats - a revolutionary blend of safety, comfort, and style. Our new car seat features innovative impact-absorbing technology, ensuring unparalleled protection for your child on every journey. With its ergonomic design and plush cushioning, it offers unmatched comfort, while the sleek and modern aesthetics make it a perfect fit for any vehicle. Upgrade to the future of car seat safety and embrace worry-free travels with our new generation car seat.


In [17]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])



Introducing the revolutionary new generation of car seat, designed to prioritize safety, comfort, and convenience for both infants and toddlers. With advanced engineering and cutting-edge technology, this car seat boasts enhanced side-impact protection, ensuring optimal safety during every journey. The ergonomic design provides unparalleled comfort, with plush cushioning and adjustable features that adapt to the child's growth and posture. This car seat also includes innovative convenience features like a quick and easy installation process, removable and machine-washable covers, and intuitive harness adjustments. Trust in this next-level car seat to provide your child with the utmost safety and comfort during every ride.


# 7. Instead of just saying what not to do, say what to do instead

In [18]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])


I'm sorry to hear that you're having trouble logging in. Could you please provide me with your username or email address associated with your account?


In [19]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])


I'm sorry to hear that you're having trouble logging into your account. Can you please provide me with the error message or any specific details about the issue you're experiencing?


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [20]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])


Sure! Here's a simple Python function that asks the user for a number in miles and converts it to kilometers:

```python
def convert_miles_to_kilometers():
    miles = float(input("Enter the number of miles: "))
    kilometers = miles * 1.60934
    print(f"{miles} miles is equal to {kilometers} kilometers.")

convert_miles_to_kilometers()
```

Here's how it works:
1. The function `convert_miles_to_kilometers` prompts the user to enter the number of miles using the `input` function. The input is converted to a float to handle decimal values.
2. The miles are then multiplied by 1.60934 to convert them to kilometers. The conversion factor is derived from the fact that 1 mile is equal to 1.60934 kilometers.
3. The result is printed using the `print` function with formatted string literals (f-strings) to display the input miles and converted kilometers.

You can call the function `convert_miles_to_kilometers` to run it and convert any number of miles to kilometers.


In [21]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers\n\
                 import ',}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])


def mile_to_km():
    miles = float(input("Enter the number of miles: "))
    kilometers = miles * 1.60934
    return kilometers

result = mile_to_km()
print("The equivalent distance in kilometers is:", result)
